### imports

In [6]:
import pandas as pd
import numpy as np
import sys
import os

# NOTE: running on Mac (first line) //  macbook (second line)
sys.path.insert(0, os.path.abspath(''))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..')))

from python_scripts.utilities.api_calls import get_cookie_from_blob, fetch_item_to_df



### loading data

In [7]:
def initial_portfolio(initial_balance, items):
    portfolio_info = {
        'initial_balance': initial_balance,
        'current_balance': initial_balance,
        'items': items,
        'allocations': {item: 1/len(items) for item in items},
        'holdings': {item: 0 for item in items},
    }
    
    item_data = {}
    dailyCookie = get_cookie_from_blob()
    for item in items:
        item_data[item] = fetch_item_to_df(item, dailyCookie)

    return portfolio_info, item_data


### input (usage)

In [8]:
# functions for simulating a portfolio

def check_thresholds(portfolio_info, item_data, date, stop_loss, gain_threshold):
    for item in portfolio_info['valid_items']:
        if date in item_data[item].index and np.isfinite(item_data[item].loc[date, 'price_usd']):
            buy_price = portfolio_info[f'{item}_buy_price']
            current_price = item_data[item].loc[date, 'price_usd']
            change_percentage = (current_price - buy_price) / buy_price
            
            if change_percentage <= -stop_loss or change_percentage >= gain_threshold:
                return True
    return False


def buy_items(portfolio_info, item_data, date):
    remaining_balance = portfolio_info['initial_balance']
    valid_items = []

    for item, allocation in portfolio_info['allocations'].items():
        if date in item_data[item].index and np.isfinite(item_data[item].loc[date, 'price_usd']):
            amount_to_spend = portfolio_info['initial_balance'] * allocation
            price = item_data[item].loc[date, 'price_usd']
            quantity = int(amount_to_spend // price)  # Integer division for whole number of items
            cost = quantity * price
            remaining_balance -= cost
            portfolio_info['holdings'][item] = quantity
            valid_items.append(item)
            portfolio_info[f'{item}_buy_price'] = price  # Store buy price for later use
        else:
            print(f"{item}: Insufficient data")

    portfolio_info['current_balance'] = remaining_balance
    portfolio_info['valid_items'] = valid_items


def sell_items(portfolio_info, item_data, date):
    total_value = portfolio_info['current_balance']
    for item in portfolio_info['valid_items']:
        if date in item_data[item].index and np.isfinite(item_data[item].loc[date, 'price_usd']):
            quantity = portfolio_info['holdings'][item]
            buy_price = portfolio_info[f'{item}_buy_price']
            sell_price = item_data[item].loc[date, 'price_usd']
            value = quantity * sell_price
            total_value += value
            net_profit = value - (quantity * buy_price)
            print(f"{item}: Bought {quantity} @ ${buy_price:.2f}, Sold @ ${sell_price:.2f}, Net: ${net_profit:,.2f}")
        else:
            print(f"{item}: Insufficient data")
    return total_value


def simulate_portfolio(initial_balance, items, start_date, end_date, stop_loss=None, gain_threshold=None):
    portfolio_info, item_data = initial_portfolio(initial_balance, items)
    
    portfolio_info['start_date'] = pd.to_datetime(start_date)
    portfolio_info['present_date'] = pd.to_datetime(end_date)
    
    print(f"\nSimulation: {portfolio_info['start_date'].date()} to {portfolio_info['present_date'].date()}")
    print(f"Initial Balance: ${portfolio_info['initial_balance']:,.2f}\n")
    
    buy_items(portfolio_info, item_data, portfolio_info['start_date'])
    
    if stop_loss is not None or gain_threshold is not None:
        date_range = pd.date_range(start=portfolio_info['start_date'], end=portfolio_info['present_date'])
        for date in date_range:
            if check_thresholds(portfolio_info, item_data, date, stop_loss, gain_threshold):
                final_value = sell_items(portfolio_info, item_data, date)
                return calculate_performance(portfolio_info, final_value)
    
    final_value = sell_items(portfolio_info, item_data, portfolio_info['present_date'])
    return calculate_performance(portfolio_info, final_value)


def calculate_performance(portfolio_info, final_value):
    initial_balance = portfolio_info['initial_balance']
    total_return = (final_value - initial_balance) / initial_balance
    days_held = (portfolio_info['present_date'] - portfolio_info['start_date']).days
    annualized_return = (1 + total_return) ** (365 / days_held) - 1

    return {
        'initial_balance': initial_balance,
        'final_value': final_value,
        'total_return': total_return,
        'annualized_return': annualized_return,
        'days_held': days_held
    }


#### input (usage)

In [9]:
# Usage
items = ["Glove Case Key", "Officer Jacques Beltram | Gendarmerie Nationale", "Kilowatt Case", "AK-47 | Blue Laminate (Factory New)", "Glove Case", "★ StatTrak™ Paracord Knife | Case Hardened (Field-Tested)"]
initial_balance = 10000
start_date = '2023-01-01'
end_date = '2024-06-01'
stop_loss_percentage = 100  # 10% stop loss
gain_threshold_percentage = 100  # 20% gain threshold

performance = simulate_portfolio(initial_balance, items, start_date, end_date, stop_loss=stop_loss_percentage, gain_threshold=gain_threshold_percentage)

print(f"\nInitial Balance: ${performance['initial_balance']:.2f}")
print(f"Final Value: ${performance['final_value']:.2f}")
print(f"Total Return: {performance['total_return']:.2%}")
print(f"Annualized Return: {performance['annualized_return']:.2%}")
print(f"Days Held: {performance['days_held']}")


Simulation: 2023-01-01 to 2024-06-01
Initial Balance: $10,000.00

Kilowatt Case: Insufficient data
★ StatTrak™ Paracord Knife | Case Hardened (Field-Tested): Insufficient data
Glove Case Key: Bought 208 @ $7.98, Sold @ $12.69, Net: $978.02
Officer Jacques Beltram | Gendarmerie Nationale: Bought 648 @ $2.57, Sold @ $9.05, Net: $4,197.10
AK-47 | Blue Laminate (Factory New): Bought 250 @ $6.65, Sold @ $11.82, Net: $1,293.00
Glove Case: Bought 687 @ $2.43, Sold @ $5.80, Net: $2,318.62

Initial Balance: $10000.00
Final Value: $18786.74
Total Return: 87.87%
Annualized Return: 56.08%
Days Held: 517
